In [ ]:


import sys
dir = '/home/vito/ibrahimm/projects/AI4Health/notebooks/ibrahimm/Generative-Models/ehr/Synthetic-data-experiments'
sys.path.append(dir)
from datetime import datetime, timedelta
import pandas as pd
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
import matplotlib.pyplot as plt
import os
import numpy as np
import torch
from sklearn.model_selection import train_test_split
# Set the CUDA device to 0
torch.cuda.set_device(0)

# Verify the current device\
current_device = torch.cuda.current_device()
print(f"Current CUDA device: {current_device}")
print(torch.cuda.is_available())

import data_access.base_loader as base_loader
import data_access.ricu_loader as ricu_loader
from absl import flags
from itables import init_notebook_mode
init_notebook_mode(all_interactive=True)





In [ ]:
# splitting parameters
train_fraction = 0.45
val_fraction = 0.1
oracle_fraction = 0
oracle_min = 100
intersectional_min_threshold = 100
intersectional_max_threshold = 1000


# # data parameters
data_name = 'mimic' # 'mimic' 'eicu'
task_name = 'mortality24' # 'aki' 'kidney_function' 'los' 'los_24' 'mortality24' 
static_var = 'ethnicity'
features = None
ricu_dataset_path = f'../../real_data/raw/{task_name}/{data_name}'
# processed_output_path = f'../../real_data/processed/{task_name}/{data_name}'
# intermed_output_path = f'../../real_data/intermed/{task_name}/{data_name}'
# processed_data_timestamp = '20250113132215'
processed_output_path = f'outputs/{task_name}/{data_name}/processed/'
intermed_output_path = f'outputs/{task_name}/{data_name}/intermed/'

seed = 0

simple_imputation = True
mode = 'raw'
# processed_data_timestamp = '20241122211819'
intermed_data_timestamp = None

standardize = False
save_intermed_data = True
save_processed_data = True
split = True
stratify =  False
intersectional = False

if split == False:
    split_text = 'No Split'

'''
Two modes of operation:

'''

loader = ricu_loader.RicuLoader(seed, task_name, data_name,static_var,ricu_dataset_path,simple_imputation,
                                    features, processed_output_path,intermed_output_path)


if mode == 'raw':
    # Create directories if they do not exist
    if save_intermed_data:
        os.makedirs(intermed_output_path, exist_ok=True)
    if save_processed_data:
        os.makedirs(processed_output_path, exist_ok=True)
        
    X_dict_tf, y_dict, static = loader.get_data(mode='raw', train_fraction=train_fraction, val_fraction=val_fraction, oracle_fraction=oracle_fraction, 
                                                oracle_min=oracle_min, intersectional_min_threshold=intersectional_min_threshold,
                                                intersectional_max_threshold=intersectional_max_threshold,
                                                standardize=standardize,
                                                stratify=stratify, intersectional=intersectional, split = split,
                                                save_intermed_data=save_intermed_data, save_processed_data=save_processed_data,
                                                demographics_to_stratify_on = ['age_group','ethnicity','gender'])
else:
    raise ValueError("Invalid mode specified. Choose 'raw', 'processed', or 'intermediate'.")

if not isinstance(X_dict_tf, dict):
    X_dict_tf = {file: X_dict_tf[file] for file in X_dict_tf.files}
    y_dict = {file: y_dict[file] for file in y_dict.files}

# data_params = {
#     'processed_data_timestamp':processed_data_timestamp,
#     'task_name': task_name,
#     'data_name': data_name,
#     'train_fraction': train_fraction,
#     'val_fraction': val_fraction,
#     'test_fraction': test_fraction,
#     'oracle_fraction': oracle_fraction,
#     'oracle_min': oracle_min,
#     'intersectional_min_threshold': intersectional_min_threshold,
#     'intersectional_max_threshold': intersectional_max_threshold,
#     'split': split_text,
#     'standardize' : standardize,
# }
X_dict_tf.keys()